In [1]:
%load_ext sql

In [2]:
%sql postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics

## SQL Subqueries

### FROM Subqueries

Calculate the average age of each athlete.

First we need to calculate the age of each athlete:

In [4]:
%%sql 
SELECT *
FROM athletes
LIMIT 5;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


index,name,country,countrycode,sport,sport_code,date_of_birth,gender
0,AAJIM Munkhbat,Mongolia,MGL,Judo,JUD,1989-01-25,Male
1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male
2,ABASLI Namig,Azerbaijan,AZE,Judo,JUD,1997-09-12,Male
3,ABASSI Mostefa,Algeria,ALG,Wheelchair Basketball,WBK,1977-09-25,Male
4,ABBAD Abderraouf,Algeria,ALG,Wheelchair Basketball,WBK,1986-10-27,Male


DATE_PART is used to return part of a date here we use it get the year

Our subquery is used to calculate the age of each athlete

In [5]:
%%sql 

SELECT 
name, 
sport,
DATE_PART('year', CURRENT_DATE) - DATE_PART('year', date_of_birth) as age
FROM athletes
LIMIT 15;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
15 rows affected.


name,sport,age
AAJIM Munkhbat,Judo,34.0
ABARZA Alberto,Swimming,39.0
ABASLI Namig,Judo,26.0
ABASSI Mostefa,Wheelchair Basketball,46.0
ABBAD Abderraouf,Wheelchair Basketball,37.0
ABDALLA Ashraf,Sitting Volleyball,49.0
ABOUELYAZEID Mohamed,Sitting Volleyball,44.0
ABDELALI Samiha,Wheelchair Basketball,35.0
ABDELAZIZ Gihan,Powerlifting,49.0
ELKHARSA Ahmed,Sitting Volleyball,26.0


Now we have the ages for each athlete we can use AVG to get the average age.

Our outer query will look like this:


```sql
SELECT subquery_result.sport, AVG(subquery_result.age) AS average_age
  FROM (...) subquery_result
  GROUP BY subquery_result.sport
  ORDER BY average_age;
```
  
  

If we combine our subquery and outer query we get:

In [6]:
%%sql

SELECT subquery_result.sport, AVG(subquery_result.age) AS average_age
  FROM (
    SELECT 
      name,
      sport,
      DATE_PART('year', CURRENT_DATE) - DATE_PART('year', date_of_birth) as age
      FROM athletes
  ) subquery_result
  GROUP BY subquery_result.sport
  ORDER BY average_age DESC;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
23 rows affected.


sport,average_age
Shooting,41.86363636363637
Equestrian,40.506493506493506
Cycling Road,39.798122065727696
Archery,39.75539568345324
Powerlifting,38.146067415730336
Sitting Volleyball,37.213903743315505
Triathlon,36.975
Table Tennis,36.780575539568346
Wheelchair Fencing,36.729166666666664
Canoe Sprint,36.43956043956044


What if we wanted to find the average age for each medal? We'd have to throw a JOIN in as well!

Let's do the subquery first.



In [8]:
%%sql

SELECT
    medals.medal,
    DATE_PART('year', CURRENT_DATE) - DATE_PART('year', athletes.date_of_birth) as age
    FROM athletes INNER JOIN medals
    ON athletes.name = medals.name
    AND athletes.sport = medals.sport
    LIMIT 15;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
15 rows affected.


medal,age
Silver Medal,39.0
Silver Medal,39.0
Gold Medal,39.0
Bronze Medal,26.0
Bronze Medal,43.0
Gold Medal,25.0
Bronze Medal,54.0
Gold Medal,54.0
Bronze Medal,38.0
Gold Medal,38.0


The outer query combined with the subquery:

In [9]:
%%sql

SELECT subquery_result.medal, AVG(subquery_result.age) as average_age
FROM (
  SELECT
    medals.medal,
    DATE_PART('year', CURRENT_DATE) - DATE_PART('year', athletes.date_of_birth) as age
    FROM athletes INNER JOIN medals
    ON athletes.name = medals.name
    AND athletes.sport = medals.sport
) subquery_result
GROUP BY medal
ORDER BY average_age DESC;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


medal,average_age
Bronze Medal,33.05710491367862
Silver Medal,32.65123010130246
Gold Medal,31.70487106017192


## WHERE Subqueries

If you wanted to get a list of all the athletes who had won a gold medal, you could use a WHERE subquery, like this


In [11]:
%%sql

SELECT name
FROM athletes
WHERE name IN (
    SELECT name
    FROM medals
    WHERE medal = 'Gold Medal'   
)
LIMIT 20;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
20 rows affected.


name
ABARZA Alberto
ABDELLAOUI Cherine
ABLINGER Walter
ABRAHAM GEBRU Daniel
ABRAHAM GEBRU Daniel
ADAMS Lisa
AFROOZ Saeid
ALARYANI Abdulla Sultan
ALCOTT Dylan
ALI Haider


What if you wanted a list of all athletes who had participated in multiple sports?

In [13]:
%%sql

SELECT name
FROM athletes
WHERE name IN (
    SELECT name
    FROM athletes
    GROUP BY name
    HAVING COUNT(DISTINCT sport) > 1
)
LIMIT 15;


 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
15 rows affected.


name
ABRAHAM GEBRU Daniel
ABRAHAM GEBRU Daniel
ASTASHOV Mikhail
ASTASHOV Mikhail
BANGMA Tristan
BANGMA Tristan
BATE Stephen
BATE Stephen
BEAUGILLET Raphael
BEAUGILLET Raphael


## JOIN Subqueries

You can also use a subquery as part of your JOIN statement if, for example, you want to manipulate data in the right hand (second) table before joining.

Here's a query that will return all the athletes who won a gold medal. It filters the medals table to get only the gold medals and then joins it to the athletes table.

In [15]:
%%sql

SELECT date_of_birth, athletes.name, athletes.sport, gold_medals.medal
FROM athletes
JOIN (
    SELECT name, event, medal
    FROM medals
    WHERE medal = 'Gold Medal'
) AS gold_medals ON athletes.name = gold_medals.name
LIMIT 15;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
15 rows affected.


date_of_birth,name,sport,medal
1984-12-11,ABARZA Alberto,Swimming,Gold Medal
1998-08-28,ABDELLAOUI Cherine,Judo,Gold Medal
1969-05-12,ABLINGER Walter,Cycling Road,Gold Medal
1985-02-11,ABRAHAM GEBRU Daniel,Cycling Road,Gold Medal
1985-02-11,ABRAHAM GEBRU Daniel,Cycling Track,Gold Medal
1990-11-18,ADAMS Lisa,Athletics,Gold Medal
1990-12-13,AFROOZ Saeid,Athletics,Gold Medal
1970-08-12,ALARYANI Abdulla Sultan,Shooting,Gold Medal
1990-12-04,ALCOTT Dylan,Wheelchair Tennis,Gold Medal
1984-12-12,ALI Haider,Athletics,Gold Medal


## SUMMARY

Subqueries can be used in FROM, WHERE and JOIN statements
 - FROM subqueries allow you to perform some initial manipulations to before filtering and / or aggregating. For example, if you wanted to derive the age of each athlete then filter for people aged 28, you'd need to use a FROM subquery.
- WHRE subqueries allow you to build dynamic conditions into your queries.
- JOIN subqueries allow you to perform initial manipulations to the right table, before joining.

## Exercise

In [16]:
%%sql

SELECT *
FROM athletes
LIMIT 10;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


index,name,country,countrycode,sport,sport_code,date_of_birth,gender
0,AAJIM Munkhbat,Mongolia,MGL,Judo,JUD,1989-01-25,Male
1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male
2,ABASLI Namig,Azerbaijan,AZE,Judo,JUD,1997-09-12,Male
3,ABASSI Mostefa,Algeria,ALG,Wheelchair Basketball,WBK,1977-09-25,Male
4,ABBAD Abderraouf,Algeria,ALG,Wheelchair Basketball,WBK,1986-10-27,Male
5,ABDALLA Ashraf,Egypt,EGY,Sitting Volleyball,VBS,1974-02-24,Male
6,ABOUELYAZEID Mohamed,Egypt,EGY,Sitting Volleyball,VBS,1979-03-29,Male
7,ABDELALI Samiha,Algeria,ALG,Wheelchair Basketball,WBK,1988-04-25,Female
8,ABDELAZIZ Gihan,Egypt,EGY,Powerlifting,PWL,1974-10-29,Female
9,ELKHARSA Ahmed,Egypt,EGY,Sitting Volleyball,VBS,1997-09-27,Male


In [26]:
%%sql

SELECT country, COUNT(country) AS athlete_count
FROM athletes
GROUP BY country
ORDER BY athlete_count DESC
LIMIT 10;


 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


country,athlete_count
Japan,262
People's Republic of China,256
RPC,248
United States of America,243
Brazil,241
Great Britain,221
Australia,181
France,146
Ukraine,139
Germany,137


In [28]:
%%sql

SELECT *
FROM medals
LIMIT 5;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
5 rows affected.


index,country,countrycode,name,sport,sport_short,event,medal
0,Czech Republic,CZE,DRAHONINSKY David,Archery,ARC,Men's Individual - W1,Gold Medal
1,People's Republic of China,CHN,HE Zihao,Archery,ARC,Men's Individual Compound - Open,Gold Medal
2,United States of America,USA,MATHER Kevin,Archery,ARC,Men's Individual Recurve - Open,Gold Medal
3,People's Republic of China,CHN,CHEN Minyi,Archery,ARC,Women's Individual - W1,Gold Medal
4,Great Britain,GBR,PATERSON PINE Phoebe,Archery,ARC,Women's Individual Compound - Open,Gold Medal


In [29]:
%%sql

SELECT country, COUNT(country) AS medal_count
FROM medals
GROUP BY country
ORDER BY medal_count DESC
LIMIT 10;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


country,medal_count
People's Republic of China,276
Great Britain,171
United States of America,167
RPC,153
Brazil,111
Ukraine,108
Australia,101
Japan,92
Italy,91
Netherlands,77


In [31]:
%%sql

SELECT
    medals_sub.country,
    athletes_sub.athlete_count,
    medals_sub.medal_count,
    ROUND(CAST(medals_sub.medal_count AS numeric) / athletes_sub.athlete_count, 2) AS   medals_per_athlete
FROM
        (SELECT country, COUNT(*) AS medal_count FROM medals GROUP BY country) AS medals_sub
JOIN
        (SELECT country, COUNT(*) AS athlete_count FROM athletes GROUP BY country) AS athletes_sub
ON
        medals_sub.country = athletes_sub.country
ORDER BY
        medals_per_athlete DESC
LIMIT 15;

 * postgresql+psycopg://localhost:5432/intro_to_sql_for_analytics
15 rows affected.


country,athlete_count,medal_count,medals_per_athlete
People's Republic of China,256,276,1.08
Netherlands,74,77,1.04
Italy,114,91,0.80
Ukraine,139,108,0.78
Great Britain,221,171,0.77
Bosnia and Herzegovina,15,11,0.73
Lithuania,11,8,0.73
Latvia,7,5,0.71
United States of America,243,167,0.69
Switzerland,21,14,0.67


The SELECT statement describes what we want as the final output, after the two subqueries and joins have been completed. 

We're after a table with 4 columns: country, athlete_count, medal_count and medals_per_athlete.

There are two subqueries - each one is used to aggreate data before the JOIN is executed. In both cases, the data are grouped by country.

Then, finally, we can join the two aggregated tables. The rest is presentational.